### Loading Packages and Data

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 8)

#import xarray as xr
import warnings
warnings.filterwarnings('ignore')
from matplotlib.backends.backend_pdf import PdfPages

# from db_queries import get_outputs as go
# from db_queries import get_ids
# from get_draws.api import get_draws
from db_queries import get_population

import altair as alt
from vega_datasets import data

In [2]:
ls /mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/19.5_51_locations/results_data

binned_ldl_exposure_time.csv     risk_exposure_time.csv
binned_ldl_exposure_time.hdf     risk_exposure_time.hdf
binned_sbp_exposure_time.csv     sbp_medication_person_time.csv
binned_sbp_exposure_time.hdf     sbp_medication_person_time.hdf
deaths.csv                       state_person_time.csv
deaths.hdf                       state_person_time.hdf
healthcare_visits.csv            transition_count.csv
healthcare_visits.hdf            transition_count.hdf
intervention_person_time.csv     ylds.csv
intervention_person_time.hdf     ylds.hdf
ldlc_medication_person_time.csv  ylls.csv
ldlc_medication_person_time.hdf  ylls.hdf


In [3]:
sim_results_dir = '/mnt/team/simulation_science/priv/engineering/vivarium_nih_us_cvd/19.5_51_locations/results_data/'

# Loading and Cleaning Data

### Load Data

In [4]:
tran = pd.read_csv(sim_results_dir + 'transition_count.csv')

state_person_time = pd.read_csv(sim_results_dir + '/state_person_time.csv')
person_time = state_person_time.loc[state_person_time['state'].isin(["acute_ischemic_stroke", "chronic_ischemic_stroke", "susceptible_to_ischemic_stroke"])].reset_index() 
person_time

index  Unnamed: 0     sex  year            measure  input_draw  \
0             0           0  Female  2021  state_person_time         829   
1             1           1  Female  2021  state_person_time         829   
2             2           2  Female  2021  state_person_time         829   
3             3           3  Female  2021  state_person_time         829   
...         ...         ...     ...   ...                ...         ...   
367196  1101596       21596    Male  2040  state_person_time         829   
367197  1101597       21597    Male  2040  state_person_time         829   
367198  1101598       21598    Male  2040  state_person_time         829   
367199  1101599       21599    Male  2040  state_person_time         829   

             scenario       age                           state      value  \
0            baseline  25_to_29           acute_ischemic_stroke   0.076660   
1       lifestyle_100  25_to_29           acute_ischemic_stroke   0.076660   
2        outreach_100  25_to_29           acute_ischemic_stroke   0.076660   
3        polypill_100  25_to_29           acute_ischemic_stroke   0.076660   
...               ...       ...                             ...        ...   
367196       baseline   95_plus  susceptible_to_ischemic_stroke  32.887064   
367197  lifestyle_100   95_plus  susceptible_to_ischemic_stroke  32.887064   
367198   outreach_100   95_plus  susceptible_to_ischemic_stroke  34.266940   
367199   polypill_100   95_plus  susceptible_to_ischemic_stroke  35.570157   

            location  
0         washington  
1         washington  
2         washington  
3         washington  
...              ...  
367196  rhode_island  
367197  rhode_island  
367198  rhode_island  
367199  rhode_island  

[367200 rows x 11 columns]

In [5]:
yll = pd.read_csv(sim_results_dir + 'ylls.csv').reset_index()
yll = yll.rename(columns = {'value':'yll'})
yld = pd.read_csv(sim_results_dir + 'ylds.csv').reset_index()
yld = yld.rename(columns = {'value':'yld', 'cause_of_disability':'cause'})
daly = yll.merge(yld, on = ['sex','year','cause','input_draw','scenario','age','location'], how='left')
daly['daly'] = daly.yll.fillna(0) + daly.yld.fillna(0)
daly = daly[['sex','year','cause','input_draw','scenario','age','location','yll','yld','daly']]
daly.head()

sex  year                  cause  input_draw       scenario       age  \
0  Female  2021  acute_ischemic_stroke         829       baseline  25_to_29   
1  Female  2021  acute_ischemic_stroke         829  lifestyle_100  25_to_29   
2  Female  2021  acute_ischemic_stroke         829   outreach_100  25_to_29   
3  Female  2021  acute_ischemic_stroke         829   polypill_100  25_to_29   
4  Female  2021  acute_ischemic_stroke         829       baseline  30_to_34   

     location  yll       yld      daly  
0  washington  0.0  0.000088  0.000088  
1  washington  0.0  0.000088  0.000088  
2  washington  0.0  0.000088  0.000088  
3  washington  0.0  0.000088  0.000088  
4  washington  0.0  0.000000  0.000000

### Load Population Data for Scaling 

In [6]:
US_pop = get_population(age_group_id=157, 
                              location_id=[102,523, 524, 525, 526, 527, 528, 529, 530,531, 532, 533,534, 535,
                                           536, 537, 538, 539, 540,541, 542, 543, 544 ,545, 546, 547, 548, 549, 550,
                                           551, 552, 553, 554, 555, 556, 557, 558, 559, 560,561, 562, 563, 564, 565, 
                                           566, 567, 568, 569, 570,571, 572, 573], 
                              year_id=2019, 
                              sex_id=3, 
                              release_id=9)
US_pop.head()

age_group_id  location_id  year_id  sex_id    population  run_id
0           157          102     2019       3  2.260218e+08     359
1           157          523     2019       3  3.425549e+06     359
2           157          524     2019       3  4.848432e+05     359
3           157          525     2019       3  4.812495e+06     359
4           157          526     2019       3  2.020596e+06     359

In [7]:
location_map = pd.read_csv('CVD_locations.csv')
US_pop = US_pop.merge(location_map,on=['location_id'])
US_pop['scalar'] = US_pop['population'] / 50_000
US_pop

age_group_id  location_id  year_id  sex_id    population  run_id  \
0            157          102     2019       3  2.260218e+08     359   
1            157          523     2019       3  3.425549e+06     359   
2            157          524     2019       3  4.848432e+05     359   
3            157          525     2019       3  4.812495e+06     359   
..           ...          ...      ...     ...           ...     ...   
48           157          570     2019       3  5.265596e+06     359   
49           157          571     2019       3  1.283409e+06     359   
50           157          572     2019       3  4.056749e+06     359   
51           157          573     2019       3  3.913808e+05     359   

    location_name abbreviation       scalar  
0              US           US  4520.435794  
1         Alabama           AL    68.510979  
2          Alaska           AK     9.696864  
3         Arizona           AZ    96.249902  
..            ...          ...          ...  
48     Washington           WA   105.311924  
49  West Virginia           WV    25.668176  
50      Wisconsin           WI    81.134990  
51        Wyoming           WY     7.827615  

[52 rows x 9 columns]

### Scale Datasets

# Create Dataset with Needed Outcomes 
•	MI, stroke, and HF incidence rates + counts 

•	IHD (MI + post-MI + HF from IHD), stroke, and HF DALYs 

•	Percent change and counts 

•	National and PDFs with states 

### Starting with Incidence Counts for MI, stroke, and HF 

In [8]:
person_time = person_time.groupby(['input_draw','year','scenario','location']).value.sum().reset_index()
person_time = person_time.rename(columns = {'value':'ptvalue'})
#person_time

In [9]:
def q25(x):
    return x.quantile(0.025)

def q975(x):
    return x.quantile(0.975)

def convert(word):
    return ' '.join(x.capitalize() or '_' for x in word.split('_'))

In [10]:
convert('new_hampshire')

'New Hampshire'

In [11]:
tran['transition_new'] = np.where(tran.transition.isin(['susceptible_to_ischemic_heart_disease_and_heart_failure_to_acute_myocardial_infarction','heart_failure_from_ischemic_heart_disease_to_acute_myocardial_infarction_and_heart_failure','post_myocardial_infarction_to_acute_myocardial_infarction']), 'Myocardial Infarction', np.where(tran.transition.isin(['susceptible_to_ischemic_stroke_to_acute_ischemic_stroke','chronic_ischemic_stroke_to_acute_ischemic_stroke']),'Ischemic Stroke','Heart Failure'))
tran_state_rate = tran.groupby(['input_draw','year','scenario','location','transition_new']).value.sum().reset_index()
tran_state_rate['location_name'] = tran_state_rate.location.apply(convert)
tran_state_rate = tran_state_rate.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time[['input_draw','year','scenario','location','ptvalue']], on=['input_draw','year','scenario','location'])
tran_state_rate['rate'] = tran_state_rate['value'] / tran_state_rate['ptvalue']
#tran_state_rate

In [12]:
tran_state_rate_baseline = tran_state_rate.loc[tran_state_rate.scenario == 'baseline']
tran_state_rate_baseline = tran_state_rate_baseline.rename(columns = {'rate':'baseline'})
tran_state_rate_total = tran_state_rate.merge(tran_state_rate_baseline[['input_draw','year','location_name','transition_new','baseline']], on=['input_draw','year','location_name','transition_new']) 
tran_state_rate_total['averted_count'] = tran_state_rate_total['baseline'] - tran_state_rate_total['rate']
tran_state_rate_total['averted_percent'] = 100*((tran_state_rate_total['baseline'] - tran_state_rate_total['rate'])/tran_state_rate_total['baseline'])
#tran_state_rate_total.head()

In [13]:
tran_state_rate_summ = (tran_state_rate_total
                      .groupby(['year','location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
#tran_state_rate_summ.head()

In [14]:
tran_state_rate_summ.columns = ["_".join(a) for a in tran_state_rate_summ.columns.to_flat_index()]
tran_state_rate_summ.reset_index(inplace=True)
tran_state_rate_summ.loc[(tran_state_rate_summ.scenario == 'baseline') & (tran_state_rate_summ.transition_new == 'Heart Failure')]
#tran_state_rate_summ.head()

year  location_name abbreviation transition_new  scenario  \
0      2021        Alabama           AL  Heart Failure  baseline   
12     2021         Alaska           AK  Heart Failure  baseline   
24     2021        Arizona           AZ  Heart Failure  baseline   
36     2021       Arkansas           AR  Heart Failure  baseline   
...     ...            ...          ...            ...       ...   
11952  2040     Washington           WA  Heart Failure  baseline   
11964  2040  West Virginia           WV  Heart Failure  baseline   
11976  2040      Wisconsin           WI  Heart Failure  baseline   
11988  2040        Wyoming           WY  Heart Failure  baseline   

       averted_count_mean  averted_count_q25  averted_count_q975  \
0                     0.0                0.0                 0.0   
12                    0.0                0.0                 0.0   
24                    0.0                0.0                 0.0   
36                    0.0                0.0                 0.0   
...                   ...                ...                 ...   
11952                 0.0                0.0                 0.0   
11964                 0.0                0.0                 0.0   
11976                 0.0                0.0                 0.0   
11988                 0.0                0.0                 0.0   

       averted_percent_mean  averted_percent_q25  averted_percent_q975  \
0                       0.0                  0.0                   0.0   
12                      0.0                  0.0                   0.0   
24                      0.0                  0.0                   0.0   
36                      0.0                  0.0                   0.0   
...                     ...                  ...                   ...   
11952                   0.0                  0.0                   0.0   
11964                   0.0                  0.0                   0.0   
11976                   0.0                  0.0                   0.0   
11988                   0.0                  0.0                   0.0   

       rate_mean  rate_q25  rate_q975  
0       0.016511  0.016511   0.016511  
12      0.010406  0.010406   0.010406  
24      0.015103  0.015103   0.015103  
36      0.016907  0.016907   0.016907  
...          ...       ...        ...  
11952   0.015596  0.015596   0.015596  
11964   0.020252  0.020252   0.020252  
11976   0.017680  0.017680   0.017680  
11988   0.016282  0.016282   0.016282  

[1000 rows x 14 columns]

### Now Making Datasets for DALYs for IHD, stroke, and heart failure

In [15]:
daly = daly.loc[daly.cause!='other_causes']
daly['transition_new'] = np.where(daly.cause.isin(['heart_failure_residual']), 'Heart Failure', np.where(daly.cause.isin(['acute_ischemic_stroke','chronic_ischemic_stroke']),'Ischemic Stroke','Ischemic Heart Disease'))
daly_state = daly.groupby(['input_draw','year','scenario','location','transition_new']).daly.sum().reset_index()
daly_state['location_name'] = daly_state.location.apply(convert)
daly_state_rate = daly_state.merge(US_pop[['location_name','abbreviation','scalar']], on=['location_name']).merge(
                             person_time[['input_draw','year','scenario','location','ptvalue']], on=['input_draw','year','scenario','location'])
daly_state_rate['rate'] = daly_state_rate['daly'] / daly_state_rate['ptvalue']
#daly_state_rate

In [16]:
daly_state

input_draw  year      scenario   location          transition_new  \
0             829  2021      baseline    alabama           Heart Failure   
1             829  2021      baseline    alabama  Ischemic Heart Disease   
2             829  2021      baseline    alabama         Ischemic Stroke   
3             829  2021      baseline     alaska           Heart Failure   
...           ...   ...           ...        ...                     ...   
12236         829  2040  polypill_100  wisconsin         Ischemic Stroke   
12237         829  2040  polypill_100    wyoming           Heart Failure   
12238         829  2040  polypill_100    wyoming  Ischemic Heart Disease   
12239         829  2040  polypill_100    wyoming         Ischemic Stroke   

              daly location_name  
0      3048.489513       Alabama  
1      5091.815043       Alabama  
2       465.119541       Alabama  
3      1885.521568        Alaska  
...            ...           ...  
12236   339.148444     Wisconsin  
12237  2846.927170       Wyoming  
12238  4637.321349       Wyoming  
12239   397.591044       Wyoming  

[12240 rows x 7 columns]

In [17]:
total = daly_state.groupby(['location', 'scenario']).daly.sum().unstack()
(total.baseline - total.outreach_100).sort_values()

location
utah           232.154044
texas          232.262416
idaho          302.323614
montana        337.104018
                  ...    
alabama        782.684771
missouri       790.320235
mississippi    815.478248
michigan       842.359736
Length: 51, dtype: float64

In [18]:
(total.baseline - total.polypill_100).sort_values()

location
utah              1975.957712
alaska            2373.128887
montana           2612.363127
california        2636.435057
                     ...     
alabama           3980.610113
oklahoma          4010.013197
missouri          4106.618738
south_carolina    4261.219957
Length: 51, dtype: float64

In [19]:
(total.baseline - total.lifestyle_100).sort_values()

location
colorado                 243.735938
maryland                 273.801640
district_of_columbia     285.788321
california               309.308755
                           ...     
tennessee                852.123031
louisiana                855.229964
kentucky                 860.547055
wyoming                 1093.122368
Length: 51, dtype: float64

In [20]:
daly_state_rate_baseline = daly_state_rate.loc[daly_state_rate.scenario == 'baseline']
daly_state_rate_baseline = daly_state_rate_baseline.rename(columns = {'rate':'baseline'})
daly_state_rate_total = daly_state_rate.merge(daly_state_rate_baseline[['input_draw','year','location_name','transition_new','baseline']], on=['input_draw','year','location_name','transition_new']) 
daly_state_rate_total['averted_count'] = daly_state_rate_total['baseline'] - daly_state_rate_total['rate']
daly_state_rate_total['averted_percent'] = 100*((daly_state_rate_total['baseline'] - daly_state_rate_total['rate'])/daly_state_rate_total['baseline'])
#daly_state_rate_total#.head()

In [21]:
daly_state_rate_summ = (daly_state_rate_total
                      .groupby(['year','location_name','abbreviation','transition_new','scenario'])
                      .aggregate({'averted_count':['mean',q25,q975],
                                  'averted_percent':['mean',q25,q975],
                                 'rate':['mean',q25,q975]}))
#daly_state_rate_summ.head()

In [22]:
daly_state_rate_summ.columns = ["_".join(a) for a in daly_state_rate_summ.columns.to_flat_index()]
daly_state_rate_summ.reset_index(inplace=True)
#daly_state_rate_summ.head()

### Converting Datasets a Bit for Maps

In [23]:
tran_state_rate_map = pd.DataFrame()
tran_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = tran_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
tran_state_rate_map_2023 = tran_state_rate_map.loc[tran_state_rate_map.year==2023].reset_index()
tran_state_rate_map_2023 = tran_state_rate_map_2023.rename(columns = {'rate_mean':'rate_mean_2023','averted_count_mean':'averted_count_mean_2023','averted_percent_mean':'averted_percent_mean_2023'}).reset_index()
tran_state_rate_map_2040 = tran_state_rate_map.loc[tran_state_rate_map.year==2040].reset_index()
tran_state_rate_map_2040 = tran_state_rate_map_2040.rename(columns = {'rate_mean':'rate_mean_2040','averted_count_mean':'averted_count_mean_2040','averted_percent_mean':'averted_percent_mean_2040'}).reset_index()
tran_state_rate_map = pd.concat([tran_state_rate_map_2023, tran_state_rate_map_2040[['rate_mean_2040','averted_count_mean_2040','averted_percent_mean_2040']]], axis=1)#.reset_index()
#tran_state_rate_map.head()

In [24]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
tran_state_rate_map = tran_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#tran_state_rate_map

In [25]:
daly_state_rate_map = pd.DataFrame()
daly_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = daly_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
daly_state_rate_map_2023 = daly_state_rate_map.loc[daly_state_rate_map.year==2023].reset_index()
daly_state_rate_map_2023 = daly_state_rate_map_2023.rename(columns = {'rate_mean':'rate_mean_2023','averted_count_mean':'averted_count_mean_2023','averted_percent_mean':'averted_percent_mean_2023'})#.reset_index()
daly_state_rate_map_2040 = daly_state_rate_map.loc[daly_state_rate_map.year==2040].reset_index()
daly_state_rate_map_2040 = daly_state_rate_map_2040.rename(columns = {'rate_mean':'rate_mean_2040','averted_count_mean':'averted_count_mean_2040','averted_percent_mean':'averted_percent_mean_2040'}).reset_index()
daly_state_rate_map = pd.concat([daly_state_rate_map_2023, daly_state_rate_map_2040[['rate_mean_2040','averted_count_mean_2040','averted_percent_mean_2040']]], axis=1)#.reset_index()
#daly_state_rate_map#_2023#.head()

In [26]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
daly_state_rate_map = daly_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])

# Make Graphs 


### 2023 vs 2040 Maps for all Outcomes

In [27]:
tran_state_rate_map_baseline = tran_state_rate_map.loc[(tran_state_rate_map.scenario == 'baseline')]
daly_state_rate_map_baseline = daly_state_rate_map.loc[(daly_state_rate_map.scenario == 'baseline')]
#tran_state_rate_map

In [28]:
def comparison_years(data_1, outcome): 
    charts = []
    for tran in data_1.transition_new.unique(): 

        subdata = data_1.loc[data_1.transition_new == tran] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['rate_mean_2023', 'rate_mean_2040']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('rate_mean_2023:Q', title= outcome),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'{tran} in 2023'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color='rate_mean_2040:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'{tran} in 2040'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        chart = alt.hconcat(chart_1, chart_2)

        charts.append(chart)
    return charts

In [29]:
incidence_charts = comparison_years(tran_state_rate_map_baseline, 'Incidence Rate')
alt.vconcat(*incidence_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

In [30]:
daly_charts = comparison_years(daly_state_rate_map_baseline, 'DALY Rate')
alt.vconcat(*daly_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

### Averted Percent by Scenario Maps

In [31]:
tran_state_rate_map = pd.DataFrame()
tran_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = tran_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
tran_state_rate_map_polypill = tran_state_rate_map.loc[tran_state_rate_map.scenario=='polypill_100'].reset_index()
tran_state_rate_map_polypill = tran_state_rate_map_polypill.rename(columns = {'rate_mean':'rate_mean_polypill','averted_count_mean':'averted_count_mean_polypill','averted_percent_mean':'averted_percent_mean_polypill'})
tran_state_rate_map_outreach = tran_state_rate_map.loc[tran_state_rate_map.scenario=='outreach_100'].reset_index()
tran_state_rate_map_outreach = tran_state_rate_map_outreach.rename(columns = {'rate_mean':'rate_mean_outreach','averted_count_mean':'averted_count_mean_outreach','averted_percent_mean':'averted_percent_mean_outreach'})
tran_state_rate_map_lifestyle = tran_state_rate_map.loc[tran_state_rate_map.scenario=='lifestyle_100'].reset_index()
tran_state_rate_map_lifestyle = tran_state_rate_map_lifestyle.rename(columns = {'rate_mean':'rate_mean_lifestyle','averted_count_mean':'averted_count_mean_lifestyle','averted_percent_mean':'averted_percent_mean_lifestyle'}).reset_index()
tran_state_rate_map = pd.concat([tran_state_rate_map_polypill, tran_state_rate_map_outreach[['rate_mean_outreach','averted_count_mean_outreach','averted_percent_mean_outreach']], tran_state_rate_map_lifestyle[['rate_mean_lifestyle','averted_count_mean_lifestyle','averted_percent_mean_lifestyle']]], axis=1)
tran_state_rate_map.head()

index  year         transition_new location_name abbreviation  \
0      3  2021          Heart Failure       Alabama           AL   
1      7  2021        Ischemic Stroke       Alabama           AL   
2     11  2021  Myocardial Infarction       Alabama           AL   
3     15  2021          Heart Failure        Alaska           AK   
4     19  2021        Ischemic Stroke        Alaska           AK   

       scenario  rate_mean_polypill  averted_count_mean_polypill  \
0  polypill_100            0.016511                          0.0   
1  polypill_100            0.002048                          0.0   
2  polypill_100            0.005836                          0.0   
3  polypill_100            0.010406                          0.0   
4  polypill_100            0.001075                          0.0   

   averted_percent_mean_polypill  rate_mean_outreach  \
0                            0.0            0.016511   
1                            0.0            0.002048   
2                            0.0            0.005836   
3                            0.0            0.010406   
4                            0.0            0.001075   

   averted_count_mean_outreach  averted_percent_mean_outreach  \
0                          0.0                            0.0   
1                          0.0                            0.0   
2                          0.0                            0.0   
3                          0.0                            0.0   
4                          0.0                            0.0   

   rate_mean_lifestyle  averted_count_mean_lifestyle  \
0             0.016511                           0.0   
1             0.002048                           0.0   
2             0.005836                           0.0   
3             0.010406                           0.0   
4             0.001075                           0.0   

   averted_percent_mean_lifestyle  
0                             0.0  
1                             0.0  
2                             0.0  
3                             0.0  
4                             0.0

In [32]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
tran_state_rate_map = tran_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#tran_state_rate_map

In [33]:
daly_state_rate_map = pd.DataFrame()
daly_state_rate_map[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']] = daly_state_rate_summ[['year','transition_new','location_name','abbreviation','scenario','rate_mean','averted_count_mean','averted_percent_mean']]
daly_state_rate_map_polypill = daly_state_rate_map.loc[daly_state_rate_map.scenario=='polypill_100'].reset_index()
daly_state_rate_map_polypill = daly_state_rate_map_polypill.rename(columns = {'rate_mean':'rate_mean_polypill','averted_count_mean':'averted_count_mean_polypill','averted_percent_mean':'averted_percent_mean_polypill'})
daly_state_rate_map_outreach = daly_state_rate_map.loc[daly_state_rate_map.scenario=='outreach_100'].reset_index()
daly_state_rate_map_outreach = daly_state_rate_map_outreach.rename(columns = {'rate_mean':'rate_mean_outreach','averted_count_mean':'averted_count_mean_outreach','averted_percent_mean':'averted_percent_mean_outreach'})
daly_state_rate_map_lifestyle = daly_state_rate_map.loc[daly_state_rate_map.scenario=='lifestyle_100'].reset_index()
daly_state_rate_map_lifestyle = daly_state_rate_map_lifestyle.rename(columns = {'rate_mean':'rate_mean_lifestyle','averted_count_mean':'averted_count_mean_lifestyle','averted_percent_mean':'averted_percent_mean_lifestyle'}).reset_index()
daly_state_rate_map = pd.concat([daly_state_rate_map_polypill, daly_state_rate_map_outreach[['rate_mean_outreach','averted_count_mean_outreach','averted_percent_mean_outreach']], daly_state_rate_map_lifestyle[['rate_mean_lifestyle','averted_count_mean_lifestyle','averted_percent_mean_lifestyle']]], axis=1)
#daly_state_rate_map

In [34]:
pop = data.population_engineers_hurricanes()
pop= pop.rename(columns = {'state':'location_name'})
daly_state_rate_map = daly_state_rate_map.merge(pop[['location_name','id']], on=['location_name'])
#daly_state_rate_map

In [35]:
pop.location_name.unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming', 'Puerto Rico'],
      dtype=object)

In [36]:
def interventions_maps(data_1, outcome): 
    charts = []
    for tran in data_1.transition_new.unique(): 

        subdata = tran_state_rate_map.loc[tran_state_rate_map.transition_new == 'Heart Failure'] 
        states = alt.topo_feature(data.us_10m.url, 'states')
        variable_list = ['averted_percent_mean_polypill', 'averted_percent_mean_outreach']

        background_1 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_1 = alt.Chart(states).mark_geoshape().encode(
            color=alt.Color('averted_percent_mean_polypill:Q', title= 'Percent Averted'),
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} {outcome} Averted with Polypill'
        ).project(
            type='albersUsa'
        )

        chart_1 = alt.layer(background_1, overlay_1)

        background_2 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_2 = alt.Chart(states).mark_geoshape().encode(
            color='averted_percent_mean_outreach:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} {outcome} Averted with Outreach'
        ).project(
            type='albersUsa'
        )

        chart_2 = alt.layer(background_2, overlay_2)

        background_3 = alt.Chart(states).mark_geoshape(
            fill='lightgray',
            stroke='white'
        ).project('albersUsa').properties(
            width=500,
            height=300
        )

        overlay_3 = alt.Chart(states).mark_geoshape().encode(
            color='averted_percent_mean_lifestyle:Q'
        ).transform_lookup(
            lookup='id',
            from_=alt.LookupData(subdata, 'id', list(subdata.columns))
        ).properties(
            width=500,
            height=300,
            title=f'Percent of {tran} Incidence Averted with Lifestyle'
        ).project(
            type='albersUsa'
        )

        chart_3 = alt.layer(background_3, overlay_3)
        chart = alt.hconcat(chart_1, chart_2, chart_3)

        charts.append(chart)
    return charts

In [37]:
incidence_averted_charts = interventions_maps(tran_state_rate_map, 'Incidence')
alt.vconcat(*incidence_averted_charts).resolve_scale(color="independent")

alt.VConcatChart(...)

In [38]:
daly_averted_charts = interventions_maps(daly_state_rate_map, 'DALYs')
alt.vconcat(*daly_averted_charts).resolve_scale(color="independent")

alt.VConcatChart(...)